In [3]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from keras import backend as K
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
%matplotlib inline

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
data_path = "/content/drive/MyDrive/Timeseries_test/data_ts.xlsx"
df = pd.read_excel(data_path,sheet_name = "AllWOs")
df.columns = df.columns.str.lower()
df['actstart'] = df['actstart'].dt.strftime("%d/%m/%Y")
df['actfinish'] = df['actfinish'].dt.strftime("%d/%m/%Y")
df=df.sort_values(by = 'actstart')
df.reset_index(drop = True,inplace = True)

In [8]:
col_usage = ['wonum','routine/non-routine','worktype','estdur','actstart','actfinish',]
df = df[df['bdpocdiscipline'] == 'MECH']
df= df[col_usage]
df['actstart'] = pd.to_datetime(df['actstart'], infer_datetime_format=True)
df['actfinish'] =pd.to_datetime(df['actfinish'], infer_datetime_format=True)
df = df.dropna()

In [9]:
data_list = {}  
worktype = df['worktype'].unique()
routine = df['routine/non-routine'].unique()

for rou in routine:
  for wtype in worktype:
    try:
      mask = (df['routine/non-routine'] == rou) & (df['worktype'] == wtype)
      CM_non_routine = df[mask]
      CM_non_routine.reset_index(drop = True,inplace = True)
      CM_non_routine = CM_non_routine.dropna()
      date_range = pd.date_range(CM_non_routine['actstart'].min(),CM_non_routine['actfinish'].max())
      df_date_range = pd.DataFrame(data=date_range)
      df_date_range['daily_est'] = 0
      df_date_range=df_date_range.set_index([0])
      for index,row in CM_non_routine.iterrows():
          range_task = pd.date_range(start= row['actstart'], end = row['actfinish'])
          if row['actfinish'] == row['actstart']:
              df_date_range.loc[row['actstart']]+=row['estdur']
          else:
              value = row['estdur']/(row['actfinish'] - row['actstart']).days
              for i in range(len(range_task)):
                  df_date_range.loc[range_task[i]]+=value
      name = str(wtype) + '_' + rou
      data_list[name] = df_date_range
    except:
      continue

In [16]:
df_all = pd.concat(data_list.values(), axis =1)
df_all = df_all.fillna(0)

In [17]:
df_all.columns = data_list.keys()

In [18]:
df_all

,PM_Routine,Routine_Routine,General_Routine,OVERHAUL_Routine,PdM_Routine,CFT_Routine,CM_Non-Routine,REC_Non-Routine,MoD_Non-Routine
0,,,,,,,,,
2013-12-14,0.202313,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2013-12-15,4.957640,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2013-12-16,4.957640,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2013-12-17,4.979667,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2013-12-18,4.979667,0.000000,0.000000,0.0,0.0,0.0,2.142857,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2021-12-25,20.000000,0.228571,15.086207,0.0,0.0,0.0,0.000000,0.0,0.0
2021-12-26,41.000000,40.228571,15.086207,0.0,0.0,0.0,0.000000,0.0,0.0
2021-12-27,20.000000,40.228571,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [20]:
df_all.to_csv('/content/drive/MyDrive/Timeseries_test/alldata.csv')  